In [1]:
import sys
from pathlib import Path
# Add project root to Python path (go up one level from notebooks/)
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
import torch
from model.model_loader import ModelLoader
from main import load_config

In [2]:
def test_model():
    """Quick test to verify model loads and runs."""
    
    # Load config
    config = load_config('configs/config.yaml')
    
    # Initialize model loader
    model_loader = ModelLoader()
    
    # Load tokenizer ONCE (it's the same for all strategies)
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(config['model']['distilbert'])
    
    # Test different freeze strategies
    print("\n=== Testing Model Loading ===")
    for strategy in ['frozen', 'partial', 'full']:
        print(f"\n--- Strategy: {strategy} ---")
        model, _ = model_loader.load_model_and_tokenizer(freeze_strategy=strategy)
        
        # Count parameters
        total, trainable = model_loader.count_parameters(model)
        print(f"Total parameters: {total:,}")
        print(f"Trainable parameters: {trainable:,}")
        print(f"Trainable percentage: {100 * trainable / total:.2f}%")
    
    # Use 'frozen' for quick test
    print("\n=== Testing Forward Pass (Untrained) ===")
    model, _ = model_loader.load_model_and_tokenizer(freeze_strategy='frozen')  # Reuse tokenizer
    model.eval()
    
    # Create dummy input
    test_texts = [
        "Apple stock rises on strong earnings",
        "Market crashes due to inflation fears",
        "Stock remains stable despite news"
    ]
    
    # Tokenize
    max_length = config['model']['max_len']  # Get from config
    inputs = tokenizer(
        test_texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    # Forward pass
    with torch.no_grad():
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        predictions = torch.argmax(logits, dim=1)
    
    print("\nPredictions (before training):")
    class_names = ['up', 'stable', 'down']
    for text, pred, logit in zip(test_texts, predictions, logits):
        print(f"Text: {text[:50]}...")
        print(f"Prediction: {class_names[pred.item()]}")
        print(f"Logits: {logit.tolist()}")
        print()
    
    # Quick training test
    print("\n=== Testing Training Step ===")
    lr = config['training']['learning_rate']
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    # Dummy labels
    labels = torch.tensor([0, 2, 1])  # up, down, stable
    
    # Single training step
    optimizer.zero_grad()
    logits = model(inputs['input_ids'], inputs['attention_mask'])
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    
    print(f"Loss after 1 step: {loss.item():.4f}")
    
    # Check if parameters updated
    model.eval()
    with torch.no_grad():
        new_logits = model(inputs['input_ids'], inputs['attention_mask'])
        new_predictions = torch.argmax(new_logits, dim=1)
    
    print("\nPredictions (after 1 training step):")
    for text, pred, logit in zip(test_texts, new_predictions, new_logits):
        print(f"Text: {text[:50]}...")
        print(f"Prediction: {class_names[pred.item()]}")
        print(f"Logits: {logit.tolist()}")
        print()

In [3]:
# Run the test
test_model()

Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml

=== Testing Model Loading ===

--- Strategy: frozen ---
Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Froze all transformer layers
Total parameters: 66,365,187
Trainable parameters: 2,307
Trainable percentage: 0.00%

--- Strategy: partial ---
Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Froze all transformer layers
Unfroze top 2 layers (layers [4, 5]) out of 6 total
Total parameters: 66,365,187
Trainable parameters: 14,178,051
Trainable percentage: 21.36%

--- Strategy: full ---
Loaded config from c:\

In [4]:
# Add this as a NEW cell after your current test_model() cell

def test_extended_training():
    """Test that model can actually learn from data."""
    
    # Load config
    config = load_config('configs/config.yaml')
    
    # Initialize model loader
    model_loader = ModelLoader()
    
    # Load tokenizer and model
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained(config['model']['distilbert'])
    model, _ = model_loader.load_model_and_tokenizer(freeze_strategy='frozen')
    
    # Create dummy input
    test_texts = [
        "Apple stock rises on strong earnings",
        "Market crashes due to inflation fears",
        "Stock remains stable despite news"
    ]
    
    # Tokenize
    max_length = config['model']['max_len']
    inputs = tokenizer(
        test_texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    # Setup training
    lr = config['training']['learning_rate']
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    labels = torch.tensor([0, 2, 1])  # up, down, stable
    
    # Train for multiple steps
    print("\n=== Testing Multiple Training Steps ===")
    for step in range(10):
        optimizer.zero_grad()
        logits = model(inputs['input_ids'], inputs['attention_mask'])
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        if (step + 1) % 2 == 0:  # Print every 2 steps
            print(f"Step {step+1}: Loss = {loss.item():.4f}")
    
    # Check final predictions
    model.eval()
    with torch.no_grad():
        final_logits = model(inputs['input_ids'], inputs['attention_mask'])
        final_preds = torch.argmax(final_logits, dim=1)
    
    print(f"\nFinal predictions: {[['up', 'stable', 'down'][p] for p in final_preds.tolist()]}")
    print(f"True labels:       {[['up', 'stable', 'down'][l] for l in labels.tolist()]}")
    print(f"Accuracy: {(final_preds == labels).sum().item()}/{len(labels)}")

# Run it
test_extended_training()

Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Loaded config from c:\Users\Besitzer\OneDrive\Dokumente\CBS_Copenhagen\Semester\WS2025\AdvNLP\Final Exam\AVDNLP_final_project\configs\config.yaml
Froze all transformer layers

=== Testing Multiple Training Steps ===
Step 2: Loss = 1.0399
Step 4: Loss = 0.9469
Step 6: Loss = 1.1083
Step 8: Loss = 0.9792
Step 10: Loss = 0.8291

Final predictions: ['up', 'down', 'stable']
True labels:       ['up', 'down', 'stable']
Accuracy: 3/3
